In [ ]:
!pip install alpha_vantage

In [ ]:
#importing modules
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from statsmodels.tsa.arima.model import ARIMA
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import mplfinance as mpf
import pandas_market_calendars as mcal  # Import the missing module
from alpha_vantage.timeseries import TimeSeries


# Download VADER for sentiment analysis
nltk.download('vader_lexicon')

ModuleNotFoundError: No module named 'vaderSentiment'

In [ ]:
#fetching historical data from Alpha Vantage

def fetch_stock_data(symbol, interval='daily', output_size='full'):
    api_key = "JO13YEMVD4ZD415D"
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}&outputsize={output_size}&datatype=json"

    response = requests.get(url).json()

    if "Time Series (Daily)" not in response:
        raise ValueError("Invalid API response. Check API Key or Symbol.")

    # Convert response to DataFrame
    df = pd.DataFrame.from_dict(response["Time Series (Daily)"], orient='index').astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    return df

# Fetch stock data
stock_symbol = "RELIANCE.NS"
df_stock = fetch_stock_data(stock_symbol)
df_stock.head()
df_stock.tail()

In [ ]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

# Replace with your actual API key
api_key = "JO13YEMVD4ZD415D"

# Format: "<symbol>.BSE" or "<symbol>.NS"
symbol = "RELIANCE.BS"  # NSE
ts = TimeSeries(key=api_key, output_format='pandas')

# Get daily data
data, meta = ts.get_daily(symbol=symbol, outputsize='full')

# Format data
data = data.rename(columns={
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. volume': 'Volume'
})

data.index = pd.to_datetime(data.index)
data = data.sort_index()
print(data.head())


ValueError: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY.

In [ ]:
#fetching latest news and perform sentiment analysis
# Replace with your actual NewsAPI key
newsapi_key = "20f7c156b4b744a48fd7dd0a971afce3"

# Stock symbol to search for
stock_symbol = "RELIANCE"

# NewsAPI endpoint for fetching news
url = f"https://newsapi.org/v2/everything?q={stock_symbol}&language=en&sortBy=publishedAt&apiKey={newsapi_key}"

# Make request
response = requests.get(url)
news_data = response.json()

# Print the response to check for errors
print(news_data)


#Step2: news extracting
# Check if 'articles' exist in response
if "articles" in news_data and len(news_data["articles"]) > 0:
    news_or_tweet_data = [article["title"] for article in news_data["articles"]]
    print("Fetched News Headlines:")
    for i, headline in enumerate(news_or_tweet_data[:5], 1):  # Show top 5 headlines
        print(f"{i}. {headline}")
else:
    print("No articles found or API error:", news_data)
    news_or_tweet_data = []  # Avoid breaking the code

# Extract date and title from news_data
articles = news_data["articles"]

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Create a list of (date, title, sentiment_score)
sentiment_data = []
for article in articles:
    date = article["publishedAt"][:10]  # Extract date in YYYY-MM-DD format
    title = article["title"]  # Use the article title as text for sentiment analysis
    sentiment_score = sia.polarity_scores(title)["compound"]  # Get sentiment score
    sentiment_data.append({"date": date, "sentiment_score": sentiment_score})

# Convert to DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Convert date column to datetime format
sentiment_df["date"] = pd.to_datetime(sentiment_df["date"])

# Group by date to get the average sentiment per day
sentiment_df = sentiment_df.groupby("date").mean().reset_index()

# Show sample sentiment data
print(sentiment_df.head())  # Debugging

#step 3: sentiment analysis:
# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis on each news headline
sentiments = [sia.polarity_scores(text)["compound"] for text in news_or_tweet_data]
sentiment_array = np.array(sentiments).reshape(-1, 1)

# Display sentiment scores
print("\nSentiment Scores:")
for i, (headline, sentiment) in enumerate(zip(news_or_tweet_data[:5], sentiments)):
    print(f"{i+1}. {headline} | Sentiment Score: {sentiment}")

{'status': 'ok', 'totalResults': 5417, 'articles': [{'source': {'id': None, 'name': 'Help Net Security'}, 'author': 'Help Net Security', 'title': 'Chinese military-linked companies dominate US digital supply chain', 'description': 'Despite growing national security concerns and government restrictions, Chinese military-linked companies remain deeply embedded in the US digital supply chain, according to Bitsight. These organizations, many of which have been designated by the US Departmen…', 'url': 'https://www.helpnetsecurity.com/2025/03/20/digital-supply-chain-security-concerns/', 'urlToImage': 'https://img.helpnetsecurity.com/wp-content/uploads/2024/12/19144006/supply_chain_1500.webp', 'publishedAt': '2025-03-20T05:30:54Z', 'content': 'Despite growing national security concerns and government restrictions, Chinese military-linked companies remain deeply embedded in the US digital supply chain, according to Bitsight. \r\nThese organiz… [+3015 chars]'}, {'source': {'id': None, 'name': '

In [ ]:
##converting sentiment scores into time series
# Convert sentiment scores into a DataFrame
sentiment_df = pd.DataFrame({
    "date": pd.to_datetime([article["publishedAt"][:10] for article in news_data["articles"]]),  # Extract date
    "sentiment_score": sentiments
})

# Group by date to get the average sentiment per day
sentiment_df = sentiment_df.groupby("date").mean().reset_index()

# Show sample sentiment data
print(sentiment_df.head())

        date  sentiment_score
0 2025-03-19         0.042954
1 2025-03-20         0.000579


In [ ]:
print(df_stock.head()

In [ ]:
df_stock.rename(columns={"index": "date"}, inplace=True)

In [ ]:
import os
import pandas as pd
import requests
import csv

session = requests.session()

headers = {
    'user-agent': "Chrome/87.0.4280.88"
}


def makeDataset(url):
    with open("dataset.csv", "w") as f:
        f.write(session.get(url).text)

    with open("dataset.csv", "r") as f:
        dataset = csv.reader(f)
        niftyData = []
        stockData = []

        for idx, row in enumerate(dataset):
            if 8 <= idx <= 67:
                niftyData.append(row)
            if 120 <= idx:
                stockData.append(row)
    os.remove("dataset.csv")
    return pd.DataFrame(niftyData), pd.DataFrame(stockData)


def getTodayData() -> object:
    webData = session.get(url="https://www.nseindia.com/api/merged-daily-reports?key=favCapital", headers=headers)
    return makeDataset(webData.json()[1]['link'])

# use it likes this
# nifty_data, companies_data = getTodayData()